In [ ]:
import pandas as pd

weibodata_1 = pd.read_csv('./weibodata.csv')
weibodata_2 = pd.read_csv('./weibodata_2.csv')
weibodata_3 = pd.read_csv('./weibodata_3.csv')
firstcolum = weibodata_1.columns.values[0]

In [ ]:
del weibodata_1[firstcolum]
del weibodata_2[firstcolum]
del weibodata_3[firstcolum]

In [ ]:
weibodata_list = [weibodata_1, weibodata_2, weibodata_3]

In [ ]:
weibodata = pd.concat(weibodata_list)

In [ ]:
weibodata.to_csv('./weibodata_final_UTF-8.csv', encoding = 'UTF-8')

In [ ]:
weibodata.to_csv('./weibodata_final_gb18030.csv', encoding = 'gb18030')

In [1]:
import pandas as pd
train_labled = pd.read_csv('./nCoV_100k_train.labled.csv', engine ='python', encoding = 'UTF-8')
test = pd.read_csv('./nCov_10k_test.csv', engine ='python', encoding = 'UTF-8')

In [2]:
train_labled = train_labled[train_labled['情感倾向'].isin(['-1','0','1'])]

In [3]:
import os
os.environ['FLAGS_eager_delete_tensor_gb'] = "0.0"
CUDA_VISIBLE_DEVICES="0" 
#显卡设置


In [4]:
#划分验证集，以8:2的比例进行划分
from sklearn.model_selection import train_test_split
train_labled = train_labled[[ '情感倾向', '微博中文内容']]
train, valid = train_test_split(train_labled, test_size=0.2, random_state=2020)
train.to_csv('./train.txt', index=False, header=False, sep='\t')
valid.to_csv('./valid.txt', index=False, header=False, sep='\t')

In [5]:
#加载自定义数据集
from paddlehub.datasets.base_nlp_dataset import TextClassificationDataset
class MyDataset(TextClassificationDataset):
    # 数据集存放目录
    base_path = './data22724/test'
    # 数据集的标签列表
    label_list=['-1', '0', '1']
    
    def __init__(self, tokenizer, max_seq_len: int = 128, mode: str = 'train'):
        if mode == 'train':
            data_file = 'train.txt'
        elif mode == 'test':
            data_file = 'valid.txt'
        else:
            data_file = 'valid.txt'
        super().__init__(
            base_path=self.base_path,
            tokenizer=tokenizer,
            max_seq_len=max_seq_len,
            mode=mode,
            data_file=data_file,
            label_list=self.label_list,
            is_file_with_header=False)

In [ ]:
import paddlehub as hub
model = hub.Module(name='ernie', task='seq-cls', num_classes=3)
tokenizer_ = model.get_tokenizer()
train_dataset = MyDataset(tokenizer=tokenizer_)
test_dataset = MyDataset(tokenizer=tokenizer_, mode='test')
dev_dataset = MyDataset(tokenizer=tokenizer_, mode='dev')

In [ ]:
import paddle
optimizer = paddle.optimizer.Adam(learning_rate=1e-4, parameters=model.parameters())
trainer = hub.Trainer(model, optimizer, checkpoint_dir='./hub_finetune', use_gpu=True)

In [ ]:
os.environ['FLAGS_eager_delete_tensor_gb'] = "0.0"
trainer.train(train_dataset, epochs=3, batch_size=12, eval_dataset=dev_dataset, save_interval=1)   # 配置训练参数，启动训练，并指定验证集


In [ ]:
result = trainer.evaluate(test_dataset, batch_size=32)  #模型评估

In [ ]:
import time
import pandas as pd
import paddlehub as hub
import numpy as np
data = pd.read_csv('./weibodata_final_UTF-8.csv', encoding='UTF-8')
# Data to be prdicted
data_content = data[['微博内容']]

In [ ]:
data_content_lists = []
for i in data_content.微博内容:
    data_content_list = []
    data_content_list.append(i)
    data_content_lists.append(data_content_list)
label_map = {-1:'negative', 0: 'neutral', 1: 'positive'}

In [ ]:
model = hub.Module(
    name='ernie',
    task='seq-cls',
    load_checkpoint='./model.pdparams',
    label_map=label_map)

In [ ]:
results = model.predict(data_content_lists, max_seq_len=128, batch_size=10, use_gpu=True)

In [ ]:
data = data_content_lists[:1000]
start = time.time()
results = model.predict(data, max_seq_len=128, batch_size=10, use_gpu=True)
end = time.time()
print(end-start)
#12.947293996810913时间测试

In [ ]:
for idx, text in enumerate(data_content_lists):
    print('Data: {} \t Lable: {}'.format(text[0], results[idx]))

In [ ]:
df = pd.DataFrame(columns=('微博内容', '情感倾向'))
for idx, text in enumerate(data_content_lists):
    df = df.append({'微博内容':text[0], '情感倾向':results[idx]}, ignore_index=True)

In [ ]:
import pandas as pd
data = pd.read_csv('./mood1.csv', encoding='UTF-8')

In [ ]:
mood_data = pd.DataFrame(columns=['时间','情感倾向'])
mood_data['时间'] = data['时间']
mood_data['情感倾向'] = data['wenben_text$weight']

In [ ]:
import datetime
timelist = []
timeformat = []
for i in mood_data['时间']:
    timelist.append(datetime.datetime.strptime(i, "%Y年%m月%d日 %H:%M"))
for i in timelist:
    timeformat.append(i.strftime("%Y-%m"))
mood_data['时间'] = timeformat
positive = mood_data[mood_data.情感倾向>0]

In [ ]:
count.keys()

In [ ]:
from collections import Counter
count_all = Counter(mood_data['时间'])
count = Counter(positive.时间)
proportion = []
for i in count.keys():
    numdata = count[i]/count_all[i]
    proportion.append(numdata)
del proportion[:2]

In [ ]:
yearlist = []
yearformat = []
for i in count.keys():
    yearformat.append(datetime.datetime.strptime(i, "%Y-%m"))
for i in yearformat:
    yearlist.append(i.strftime("%Y"))
del yearlist[:2]
for i in range(135):
    yearlist[i] = int(yearlist[i])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure,show
s = pd.Series(proportion)
s.iloc[0] = 0
s.iloc[-1] = 0
v = pd.Series(yearlist)
# 创建数据
p = figure(plot_width=1000, plot_height=400)
p.patch(v.index, s.values,    # 设置x，y值
        line_width=1, line_alpha = 0.8, line_color = 'black',line_dash = [10,4],   # 线型基本设置
        fill_color = 'black',fill_alpha = 0.2
        )
# 绘制面积图
# .patch将会把所有点连接成一个闭合面

show(p)

In [ ]:
v = pd.Series(yearlist)
v

In [ ]:
s